In [1]:
import os
import glob
import random

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data import Dataset

import pytorch_lightning as pl
from pytorch_lightning import loggers
import numpy as np

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw

In [2]:
mean = 0
var = 5
sigma = var ** 0.5
gaussian = np.random.normal(mean, sigma, (28, 28))
fonts_folder = "D:/projects/computer_vision/sudoku_solver/fonts/"
batch_sz = 500
n_iters = 2500
features_train = 100000
#num_epochs = int(n_iters / (features_train / batch_sz))
num_epochs = 5
loss_fn = nn.CrossEntropyLoss()
transform = transforms.Compose([transforms.Resize((28, 28)), transforms.ToTensor(),
                                transforms.Normalize((0.5, ), (0.5, ))])

In [3]:
def get_noise():
    mean = random.randint(0, 2)
    var = random.randint(0, 11)
    sigma = var ** 0.5
    gaussian = np.random.normal(mean, sigma, (28, 28))
    return gaussian

In [4]:
class PrintedMNIST(Dataset):
    """Generates images containing a single digit from font"""

    def __init__(self, N, random_state, transform=None):
        """"""
        self.N = N
        self.random_state = random_state
        self.transform = transform

        #fonts_folder = "fonts"

        self.fonts = [fonts_folder + "helvetica_bold1.ttf", fonts_folder + 'AovelSansRounded-rdDL.ttf', fonts_folder + 'Paul-le1V.ttf']
        #self.fonts = glob.glob(fonts_folder + "/*.ttf")

        random.seed(random_state)

    def __len__(self):
        return self.N

    def __getitem__(self, idx):

        target = random.randint(0, 9)
        color = 0
        # Generate image
        img = Image.new("L", (256, 256))
        img = np.array(img)
        img[img == 0] = 225
        img = Image.fromarray(img)

        size = 200
        x = 20
        y = 20

        draw = ImageDraw.Draw(img)
        font = ImageFont.truetype(random.choice(self.fonts), size)
        draw.text((x, y), str(target), color, font=font)
        shape = [(0, 0), (256 - 10, 256 - 10)]
        draw.rectangle(shape, outline ="black", width=4)

        img = img.resize((28, 28), Image.BILINEAR)
        #gaussian = get_noise()
        noisy_image = img + gaussian
        noisy_image = Image.fromarray(noisy_image)
        if self.transform:
            noisy_image = self.transform(noisy_image)
            #img = self.transform(img)

        return noisy_image, target

In [5]:
class CNNModel(pl.LightningModule):

    def __init__(self, classes=10):
        super().__init__()
        self.save_hyperparameters()
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=0)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(64 * 4 * 4, classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        batch_size, channels, width, height = x.size()
        x = self.cnn1(x)
        x = F.relu(x)
        x = self.maxpool1(x)
        x = self.cnn2(x)
        x = F.relu(x)
        x = self.maxpool2(x)
        x = x.view(batch_size, -1)
        x = self.fc1(x)
        out = self.softmax(x)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = loss_fn(y_hat, y)
        y_hat_pred = self.flatten(y_hat.argmax(axis=1)).float().requires_grad_(True)
        correct = (y == y_hat_pred).sum()
        logs = {'train_loss': loss}
        return {'loss': loss, 'log': logs, 'correct': correct, 'total': len(y)}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        y_hat_pred = self.flatten(y_hat.argmax(axis=1)).float().requires_grad_(True)
        loss = loss_fn(y_hat, y)
        correct = (y == y_hat_pred).sum()
        logs = {'val_loss': loss}
        return {'loss': loss, 'log': logs, 'correct': correct, 'total': len(y)}
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        correct = sum([x["correct"] for  x in outputs])
        total = sum([x["total"] for  x in outputs])
        self.log("train_loss", avg_loss, prog_bar=True, logger=True)
        self.log("train_acc", correct/total, prog_bar=True, logger=True)
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        correct = sum([x["correct"] for  x in outputs])
        total = sum([x["total"] for  x in outputs])
        self.log("val_loss", avg_loss, prog_bar=True, logger=True)
        self.log("val_acc", correct/total, prog_bar=True, logger=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def train_dataloader(self):
        train_set = PrintedMNIST(100000, -666, transform)
        train_loader = DataLoader(train_set, batch_size=batch_sz, shuffle=False, num_workers=0, drop_last=True)
        return train_loader

    def val_dataloader(self):
        val_set = PrintedMNIST(5000, 33, transform)
        val_loader = DataLoader(val_set, batch_size=batch_sz, num_workers=0)
        return val_loader

    def flatten(self, t):
        t = t.reshape(1, -1)
        t = t.squeeze()
        return t

In [6]:
tb_logger = loggers.TensorBoardLogger('logs/')
model = CNNModel()
trainer = pl.Trainer(gpus=1, max_epochs=num_epochs, logger=tb_logger, checkpoint_callback=False)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [7]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type      | Params
---------------------------------------
0 | cnn1     | Conv2d    | 832   
1 | maxpool1 | MaxPool2d | 0     
2 | cnn2     | Conv2d    | 51.3 K
3 | maxpool2 | MaxPool2d | 0     
4 | fc1      | Linear    | 10.2 K
5 | softmax  | Softmax   | 0     
---------------------------------------
62.3 K    Trainable params
0         Non-trainable params
62.3 K    Total params
0.249     Total estimated model params size (MB)


Validation sanity check:   0%|                                                                                                                                                            | 0/2 [00:00<?, ?it/s]

D:\Application\miniconda\envs\opencv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:106: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
D:\Application\miniconda\envs\opencv\lib\site-packages\torchvision\transforms\functional.py:75: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  img

D:\Application\miniconda\envs\opencv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:106: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Epoch 0:   0%|▋                                                                                                                                            | 1/210 [00:01<02:06,  1.65it/s, loss=2.34, v_num=27]

D:\Application\miniconda\envs\opencv\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\result.py:398: LightningDeprecationWarning: One of the returned values {'correct', 'total', 'log'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  f"One of the returned values {set(extra.keys())} has a `grad_fn`. We will detach it automatically"


Epoch 0:  95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍      | 200/210 [02:08<00:06,  1.56it/s, loss=2.37, v_num=27]
Validating: 0it [00:00, ?it/s]
Epoch 1:  96%|█████████████████████████████████████████████████████████████████████▊   | 201/210 [02:07<00:05,  1.59it/s, loss=2.36, v_num=27, val_loss=2.360, val_acc=0.104, train_loss=2.370, train_acc=0.090]
Validating: 0it [00:00, ?it/s]
Epoch 2:  11%|███████▉                                                                 | 23/210 [00:14<01:55,  1.61it/s, loss=2.36, v_num=27, val_loss=2.360, val_acc=0.0992, train_loss=2.370, train_acc=0.096]

D:\Application\miniconda\envs\opencv\lib\site-packages\pytorch_lightning\trainer\trainer.py:1047: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [9]:
import cv2
import numpy as np
img = cv2.imread('D:/projects/computer_vision/sudoku_solver/cell.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (28, 28))
img = Image.fromarray(img)
img = transform(img)
img = img.reshape(1, 1, 28, 28)
predictions = model(img.float())
predictions = predictions.detach().numpy()
print(np.argmax(predictions))

4


In [12]:
predictions

array([[2.6831208e-04, 7.4046344e-02, 3.1547007e-01, 1.2510201e-01,
        2.6204610e-01, 4.9022799e-03, 4.0500995e-04, 1.8129952e-01,
        9.9457242e-03, 2.6514661e-02]], dtype=float32)

In [26]:
img = x[10][0].numpy()
img = Image.fromarray(img)
img = transform(img)
img = img.reshape(1, 1, 28, 28)
predictions = model(img.float())
predictions = predictions.detach().numpy()
print(np.argmax(predictions))

6


In [28]:
y[10]

tensor(6)

In [30]:
noisy_image = Image.fromarray(x[10][0].numpy())

In [32]:
noisy_image.convert('RGB')